##### my-zero-to-gbm-proj-assign

In [1]:
import pandas as pd
import numpy as np

In [2]:
inputs = pd.read_csv('inputs.csv',index_col=0)
targets = pd.read_csv('targets.csv',index_col=0)

In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
from sklearn.metrics import r2_score
# def rmspe(y_true, y_pred):
#     return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [5]:
from sklearn.model_selection import ShuffleSplit, RepeatedKFold

In [6]:
# ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=11111)
ss = RepeatedKFold(n_splits=3, n_repeats=3, random_state=11111)

In [7]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3)
poly_inputs = pd.DataFrame(poly.fit_transform(inputs)).astype('float32')

In [8]:
from sklearn.metrics import mean_squared_error

In [9]:
from lightgbm import LGBMRegressor

In [10]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [11]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import HalvingGridSearchCV

In [12]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
    model = LGBMRegressor(random_state=11111,n_jobs=-1,silent=False,metric='rmse',force_col_wise=True,**params)
    model.fit(X_train, train_targets)#,early_stopping_rounds=5,eval_set=(X_val,val_targets))
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    train_R2 = round(r2_score(train_targets,pred_train ),3)
    train_RMSPE = round(mean_squared_error(train_targets, pred_train,squared=False),3)
    val_R2 = round(r2_score(val_targets, pred_val),3)
    val_RMSPE = round(mean_squared_error(val_targets, pred_val,squared=False),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

In [13]:
modelc = LGBMRegressor(random_state=11111,n_jobs=-1,silent=False,metric='rmse',force_col_wise=True)#,bagging_freq= 5, bagging_fraction= 0.75)
param_grid = {"boosting_type": ['gbdt','goss','dart'],
              "learning_rate" : [0.1,0.01,0.001],
              "n_estimators": [100,300,500],
              "min_child_samples": [10,20,30],
              "min_split_gain": [0.0,0.001,0.1],
            }

In [14]:
models = []
model_params = []
grid_params = []
for train_idxs, val_idxs in ss.split(poly_inputs):
   X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
   X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
   X_train = np.ascontiguousarray(X_train).reshape(-1,10)
   train_targets = np.ascontiguousarray(train_targets).ravel()
   X_val = np.ascontiguousarray(X_val).reshape(-1,10)
   val_targets = np.ascontiguousarray(val_targets).ravel()
   grid_search = HalvingGridSearchCV(modelc, param_grid,n_jobs=-1,verbose=1,return_train_score=True,random_state=11111,factor=3,min_resources=1000,error_score='raise').fit(X_train, train_targets)
   model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                   train_targets, 
                                                   X_val, 
                                                   val_targets, 
                                                   **grid_search.best_params_)
   models.append(model)
   model_params.append(model.get_params())
   grid_params.append(grid_search.best_params_)
   print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 6
min_resources_: 1000
max_resources_: 285954
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 243
n_resources: 1000
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
----------
iter: 1
n_candidates: 81
n_resources: 3000
Fitting 5 folds for each of 81 candidates, totalling 405 fits
----------
iter: 2
n_candidates: 27
n_resources: 9000
Fitting 5 folds for each of 27 candidates, totalling 135 fits
----------
iter: 3
n_candidates: 9
n_resources: 27000
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 4
n_candidates: 3
n_resources: 81000
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 285954, number of used features: 9
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 0.003881
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of 

In [ ]:
modelc.feature_importances_

In [ ]:
df_gd_par = pd.DataFrame(grid_params)
df_md_par = pd.DataFrame(model_params)

In [ ]:
df_gd_par

In [ ]:
df_md_par